<a href="https://colab.research.google.com/github/NazarioR9/BNBR_Challenge/blob/master/mask_language_modeling/MLM_BertBase_finetuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# References

In [ ]:
# How to Fine-Tune BERT for Text Classification? (https://arxiv.org/pdf/1905.05583.pdf)
# https://github.com/huggingface/blog/blob/master/notebooks/01_how_to_train.ipynb

# Installs

In [ ]:
!pip install transformers --quiet

     |████████████████████████████████| 675kB 2.8MB/s 
     |████████████████████████████████| 890kB 12.3MB/s 
     |████████████████████████████████| 1.1MB 16.6MB/s 
     |████████████████████████████████| 3.8MB 26.8MB/s 


In [ ]:
# Check that we have a GPU
!nvidia-smi

Fri Jun 26 18:39:22 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.36.06    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P0    25W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

#Imports

In [ ]:
import os, sys, gc

In [ ]:
import pandas as pd

In [ ]:
import torch
from transformers import BertConfig, BertTokenizerFast, BertForMaskedLM
from transformers import Trainer, TrainingArguments
from transformers import DataCollatorForLanguageModeling

In [ ]:
from transformers import pipeline

In [ ]:
sys.path.insert(0, '../bnbr')

In [ ]:
from bnbr.utils import seed_everything
from bnbr.data import MentalHealthDataset

Using TensorFlow backend.


# Envs

In [ ]:
seed_everything(2020)

Set seed to 2020.


In [ ]:
!mkdir ../mlm_finetuned_models

# Utils

In [ ]:
class MLMDataset(MentalHealthDataset):
  def __init__(self, df, tokenizer):
    super(MLMDataset, self).__init__(df, task='test')
    self.tokenizer = tokenizer

  def __getitem__(self, idx):
    text, _, _ = super().__getitem__(idx)
    enc = self.tokenizer.encode(text)
    return torch.tensor(enc)

#Read 

In [ ]:
path = '../data/'

In [ ]:
train = pd.read_csv(path+'final_train.csv')
test = pd.read_csv(path+'final_test.csv')

In [ ]:
train['length'] = 0
test['length'] = 0

In [ ]:
df = pd.concat([train, test])

In [ ]:
df.head()

,ID,text,label,Depression,Alcohol,Suicide,Drugs,length
0,SUAVK39Z,i feel that it was better i die am happy,0.0,1.0,0.0,0.0,0.0,0
1,9JDAGUV3,why do i get hallucinations ?,3.0,0.0,0.0,0.0,1.0,0
2,419WR1LQ,i am stressed due to lack of financial suppor...,0.0,1.0,0.0,0.0,0.0,0
3,6UY7DX6Q,why is life important ?,2.0,0.0,0.0,1.0,0.0,0
4,FYC0FTFB,how could i be helped to go through the depre...,0.0,1.0,0.0,0.0,0.0,0


#Finetuning MLM

In [ ]:
model_name = 'bert-base-uncased'

In [ ]:
config = BertConfig.from_pretrained(model_name)
tokenizer = BertTokenizerFast.from_pretrained(model_name, model_max_length =50)

In [ ]:
model = BertForMaskedLM.from_pretrained(model_name)
# model = BertForMaskedLM(config=config)

In [ ]:
dataset = MLMDataset(df[['text', 'length']], tokenizer)

In [ ]:
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

In [ ]:
training_args = TrainingArguments(
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=64,
    save_steps=10_000,
    save_total_limit=2,
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
    prediction_loss_only=True,
)

### Start training

In [ ]:
%%time
trainer.train()



CPU times: user 8.98 s, sys: 2.9 s, total: 11.9 s
Wall time: 12.2 s


TrainOutput(global_step=45, training_loss=2.5907892730500963)

#### 🎉 Save final model

In [ ]:
!mkdir fms

In [ ]:
trainer.save_model("fms/")

In [ ]:
!zip -r mlm_bert_base_.zip fms/

  adding: mlm_finetuned/ (stored 0%)
  adding: mlm_finetuned/pytorch_model.bin (deflated 8%)
  adding: mlm_finetuned/config.json (deflated 47%)
  adding: mlm_finetuned/training_args.bin (deflated 36%)


In [ ]:
!rm -r fms/